In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.ensemble import GradientBoostingClassifier
import seaborn as sns
from scipy.optimize import minimize
from scipy import optimize
import datetime as dt
from sklearn.preprocessing import MinMaxScaler

import gc

for root, dirs, files in os.walk('/kaggle/input'):
    for file in files:
        if file.endswith(".csv"):
            print(os.path.join(root, file))

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# train.event_data.str.findall(r"duration': '(.+?)'")


In [ ]:
pd.options.display.max_colwidth = 200

sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
train_labels =  pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')

specs = reduce_mem_usage(specs)
train_labels = reduce_mem_usage(train_labels)
test = reduce_mem_usage(test)
train = reduce_mem_usage(train)


# media_sequence = pd.read_csv("../input/dsb2019-external-data/media_sequence.csv")
test['timestamp'] = pd.to_datetime(test['timestamp'])
test['timestamp'] = pd.to_datetime(test['timestamp'].values)#using .'values' will remove tzinfo
train['timestamp'] = pd.to_datetime(train['timestamp'])
train['timestamp'] = pd.to_datetime(train['timestamp'].values)

# specs to DF
a = []


for i in range(len(specs.args)):
    lst = json.loads(specs['args'][i])
    args = pd.DataFrame(lst)
    args['event_id'] = specs.event_id[i]
    args['desc'] = specs['info'][i]
    a.append(args)
    
specs = pd.concat(a)

In [ ]:

# filtered_train = train[train.type == "Assessment"]
# print(test.installation_id.nunique()) # 1000 unique installation id in test
# print(sum(test.groupby(['installation_id'])['game_session'].nunique() ==1)) # 17 installation ids for wich there is no previous assessment data
# print(train.installation_id.nunique()) #17000 in train #4242 with assessments in train
# print(sum(train.groupby(['installation_id'])['game_session'].nunique() ==1) ) #1463 have only one assessment
# print(sum(train.installation_id.isin(test.installation_id.unique()))) # No test installation id is in train

# filtered_train[['title','world']].drop_duplicates() # world > title hierarchy

### assesments are not repeated after clearing it in the same session
# train_labels[(train_labels.accuracy == 0.5) & (train_labels.accuracy_group != 2)]

# print(sum(train.event_data.str.contains('"correct":true')))
# print(sum(train.event_data.str.contains('"correct":false')))
# train[train.event_data.str.contains('"correct":false')].groupby('type').count()
# # train.groupby(['installation_id','game_session'])



In [ ]:
media_sequence = pd.read_csv("../input/dsb2019-external-data/media_sequence.csv")
media_sequence = reduce_mem_usage(media_sequence)
media_sequence = media_sequence.merge(train[['world','title']].drop_duplicates(),on = 'title',how = 'left')
media_sequence['sequence'] = pd.Series(media_sequence.index)
media_sequence['world_sequence'] = media_sequence.groupby(['world'])['sequence'].rank().astype('int')
media_sequence['world_type_sequence'] = media_sequence.groupby(['world','type'])['sequence'].rank().astype('int')
media_sequence = media_sequence.drop(['type','world','sequence'],axis = 1).rename(columns = {'duration':'clip_length'})
media_sequence['clip_length'] = media_sequence['clip_length']*1000


train = train.merge(media_sequence,on  = ['title'],how = 'left')
test = test.merge(media_sequence,on  = ['title'],how = 'left')

In [ ]:
# %%time
# event_id_code_mapped = pd.concat([train[['event_id','event_code']].drop_duplicates(),test[['event_id','event_code']].drop_duplicates()])\
#                          .drop_duplicates()
                                 
# args_by_event = specs[['name','type','event_id']].merge(event_id_code_mapped, on = ['event_id'],how = 'left')\
#                                                  .drop(['event_id'],axis = 1).drop_duplicates()
# event_codes = args_by_event.event_code.unique()

# a = []

# for event in event_codes:
#     sub = train[train.event_code == event]
#     print(str(event) + '---' +str(sub.shape))
#     args = args_by_event[args_by_event.event_code == event]['name']
#     args = list(set(args) - set(['event_count','event_code','game_time','description']))
#     sub1  = sub['event_data'].apply(json.loads).apply(pd.Series)
#     a.append(reduce_mem_usage(sub1))
#     print(args)
    


In [ ]:
# Collect previous events data for each installation_id and game_session
# Create features at an installation_id + game_session
# for any given installation_id + game_session, collect all the interaction info before current game_session
# Consider all train instead of only Assesmyent types for pre data
# rank game_sessions per installation_id in chronological order
# accuracies in each game/ assessment
# installation_id, game_session, title, world, avg_time_prev, avg_events_prev, avg_clip_time_prev, avg_game_time_prev, avg_accuracy_prev, 
def fetch_column_from_json( df,extract_key, text_col_name = 'event_data',dtype = 'int' ):
    find_string = str('"')+ extract_key + str('":')
    
    exists = df[text_col_name].str.find(find_string) != -1
    df['temp'] = ''
    df['temp'][exists]=  df[text_col_name].apply(lambda x : x[x.find(find_string) + len(find_string) : len(x)-1])
    
    if (dtype == 'str') : 
        df[extract_key] = ''
    else :
        df[extract_key] = np.NaN
        
    df[extract_key][exists]= df['temp'][exists].apply(lambda x : x[:x.find(',')]).astype(dtype)

    df = df.drop(['temp'],axis = 1)
    return(df)


def extract_features (df):
    
    # all events with accuracy data in "event data json"
    # df = fetch_column_from_json(df, 'correct', dtype = 'str')
# df = fetch_column_from_json(df, 'total_duration')
# # df = fetch_column_from_json(df, 'duration')
# df = fetch_column_from_json(df, 'round')
# df = fetch_column_from_json(df, 'media_type',dtype = 'str')
# df = fetch_column_from_json(df, 'correct',dtype = 'str')
# df = fetch_column_from_json(df, 'misses',dtype = 'int')
# df = fetch_column_from_json(df, 'dwell_time',dtype = 'int')

    df['accurate'] = 0
    df.loc[df.event_data.str.contains('"correct":true'),'accurate'] = 1
    df['inaccurate'] = 0
    df.loc[df.event_data.str.contains('"correct":false'),'inaccurate'] = 1 

    # only final accuracy for calculating train labels
    df['num_correct'] = 0
    df.loc[((df.event_data.str.contains('"correct":true')) & (df.event_code .isin([4100,4110]))),'num_correct'] = 1
    df['num_incorrect'] = 0
    df.loc[((df.event_data.str.contains('"correct":false'))& (df.event_code .isin([4100,4110]))),'num_incorrect'] = 1 


    df['prev_game_time'] = df.groupby(['installation_id','game_session'])['game_time'].shift(1)
    df['game_time'] = df['game_time'] - df['prev_game_time']
    df = df.drop(['prev_game_time'],axis = 1)

    df = df.join(pd.get_dummies(df.event_code))
    reduce_mem_usage(df)


    grouped_by_session = df.groupby(['installation_id','game_session'])

    session_info = df[['installation_id','game_session','type','world','title','clip_length','world_sequence']].drop_duplicates()
    #add worldtypesequence


    ####################################################
    ###extract time features
    session_start = grouped_by_session['timestamp'].min().reset_index().rename(columns = {'timestamp': 'start_time'})
    session_end = grouped_by_session['timestamp'].max().reset_index().rename(columns = {'timestamp': 'end_time'})
    session_times = session_start.merge(session_end,on = ['installation_id','game_session'])\
                            .sort_values(['installation_id','start_time'])
    session_info = session_info.merge(session_times,on = ['installation_id','game_session'],how= 'left')
    session_info['prev_start_time'] = session_info.groupby(['installation_id'])['start_time'].shift(1)
    session_info['prev_end_time'] = session_info.groupby(['installation_id'])['end_time'].shift(1)
    session_info['gap_time'] = (session_info['start_time'] - session_info['prev_end_time']).astype('timedelta64[ms]')
    session_info['game_time'] = (session_info['end_time']- session_info['start_time']).astype('timedelta64[ms]')
    session_info = session_info.drop(['prev_start_time','prev_end_time'], axis = 1)
    session_info['day_of_week'] = session_info['start_time'].dt.day_name()
    session_info['hour_of_day'] = session_info['start_time'].dt.hour
    # order sessions by chronological order
    session_info['rank'] = session_info.groupby(['installation_id'])['start_time'].apply(lambda x: x.rank())


    ##########################################
    event_code_cols = pd.get_dummies(df.event_code).columns
    for event_code in event_code_cols:
        df[event_code] = df[event_code]*df['game_time']
        event_code_info = grouped_by_session[event_code].sum().reset_index()
        session_info = session_info.merge(event_code_info, on = ['installation_id','game_session'],how = 'left')

    del df
    gc.collect()
    # count of events, total session time and accuracy by each session 
    
    event_counts =  grouped_by_session['event_count'].max().reset_index()
    session_info = session_info.merge(event_counts,on = ['installation_id','game_session'] , how = 'left')
    
    accuracy =  grouped_by_session['accurate','inaccurate','num_correct','num_incorrect'].sum().reset_index()
    session_info = session_info.merge(accuracy, on = ['installation_id','game_session'],how = 'left')


#     session_info['clips_perc_watched'] = session_info['actual_duration']/session_info['clip_length']

    # print (sum(session_info.game_time[session_info.type == 'Clip'])) # No gametime available on clips

    session_info = session_info.sort_values(['installation_id','start_time'])

    onehot_type = pd.get_dummies(session_info.type)
    session_info = session_info.join(onehot_type)

    onehot_world = pd.get_dummies(session_info.world)
    session_info = session_info.join(onehot_world)

    session_info['game_event_count'] = session_info.event_count * session_info.Game
    session_info['activity_event_count'] = session_info.event_count * session_info.Activity
    session_info['assessment_event_count'] = session_info.event_count * session_info.Assessment
    session_info['clip_event_count'] = session_info.event_count * session_info.Clip

    session_info['game_session_time'] = session_info.game_time * session_info.Game
    session_info['activity_session_time'] = session_info.game_time * session_info.Activity
    session_info['assessment_session_time'] = session_info.game_time * session_info.Assessment
    session_info['clip_session_time'] = session_info.game_time * session_info.Clip

    session_info['game_accurate'] = session_info.accurate * session_info.Game
    session_info['assessment_accurate'] = session_info.accurate * session_info.Assessment
    session_info['game_inaccurate'] = session_info.inaccurate * session_info.Game
    session_info['assessment_inaccurate'] = session_info.inaccurate * session_info.Assessment

   

    def get_pre_data(colname,df):
        new_colname = "prev_"+ str(colname)
        df[new_colname] = (df.groupby(['installation_id'])[colname].cumsum()) - (df[colname])
        df = df.drop(colname,axis =1)
        print(new_colname)
        return(df)


    for event_code in event_code_cols:
        session_info = get_pre_data(event_code,session_info)
        
#    session_info = get_pre_data('clip_length',session_info)
#     session_info = get_pre_data('clip_session_time',session_info)
#     session_info['clips_fraction_watched'] = session_info['prev_clip_session_time']/session_info['prev_clip_length']
#     session_info['clips_fraction_watched'][session_info['clips_fraction_watched'] .isna()] = 0 

    
    #Collect previous events data for each installation_id and game_session

    

    session_info['prev_event_count'] = (session_info.groupby(['installation_id'])['event_count'].cumsum()) - (session_info.event_count)
    session_info['prev_game_time'] = (session_info.groupby(['installation_id'])['game_time'].cumsum()) -(session_info.game_time)
    session_info['prev_accurate'] = (session_info.groupby(['installation_id'])['accurate'].cumsum()) - (session_info.accurate)
    session_info['prev_inaccurate'] = (session_info.groupby(['installation_id'])['inaccurate'].cumsum()) - (session_info.inaccurate)

    session_info['prev_clip'] = (session_info.groupby(['installation_id'])['Clip'].cumsum()) - (session_info.Clip)
    session_info['prev_activity'] = (session_info.groupby(['installation_id'])['Activity'].cumsum()) - (session_info.Activity)
    session_info['prev_game'] = (session_info.groupby(['installation_id'])['Game'].cumsum()) - (session_info.Game)
    session_info['prev_assessment'] = (session_info.groupby(['installation_id'])['Assessment'].cumsum()) - (session_info.Assessment)

    session_info['prev_session_count'] = session_info['prev_clip']+session_info['prev_activity'] + \
                                            session_info['prev_game']+session_info['prev_assessment']

    session_info['prev_assessment_game_time'] = (session_info.groupby(['installation_id'])['assessment_session_time'].cumsum()) - (session_info.assessment_session_time)
    session_info['prev_assessment_event_count'] = (session_info.groupby(['installation_id'])['assessment_event_count'].cumsum()) - (session_info.assessment_event_count)
    session_info['prev_assessment_accurate'] = (session_info.groupby(['installation_id'])['assessment_accurate'].cumsum()) - (session_info.assessment_accurate)
    session_info['prev_assessment_inaccurate'] = (session_info.groupby(['installation_id'])['assessment_inaccurate'].cumsum()) - (session_info.assessment_inaccurate)

    session_info['prev_game_game_time'] = (session_info.groupby(['installation_id'])['game_session_time'].cumsum()) - (session_info.game_session_time)
    session_info['prev_game_event_count'] = (session_info.groupby(['installation_id'])['game_event_count'].cumsum()) - (session_info.game_event_count)
    session_info['prev_game_accurate'] = (session_info.groupby(['installation_id'])['game_accurate'].cumsum()) - (session_info.game_accurate)
    session_info['prev_game_inaccurate'] = (session_info.groupby(['installation_id'])['game_inaccurate'].cumsum()) - (session_info.game_inaccurate)

    session_info['prev_activity_game_time'] = (session_info.groupby(['installation_id'])['activity_session_time'].cumsum()) - (session_info.activity_session_time)
    session_info['prev_activity_event_count'] = (session_info.groupby(['installation_id'])['activity_event_count'].cumsum()) - (session_info.activity_event_count)



    session_info = session_info[session_info.type == 'Assessment']

    session_info['prev_num_correct'] = (session_info.groupby(['installation_id'])['num_correct'].cumsum()) - (session_info.num_correct)
    session_info['prev_num_incorrect'] = (session_info.groupby(['installation_id'])['num_incorrect'].cumsum()) - (session_info.num_incorrect)

    session_info['prev_mean_accuracy'] = session_info['prev_num_correct']/(session_info['prev_num_correct']+session_info['prev_num_incorrect'])
    session_info['prev_mean_accuracy'][session_info['prev_mean_accuracy'] .isna()]= 0

    session_info['prev_game_accuracy'] = session_info['prev_game_accurate']/(session_info['prev_game_accurate']+session_info['prev_game_inaccurate'])
    session_info['prev_game_accuracy'][session_info['prev_game_accuracy'] .isna()] = 0

    session_info['prev_assessment_accuracy'] = session_info['prev_assessment_accurate']/(session_info['prev_assessment_accurate']+session_info['prev_assessment_inaccurate'])
    session_info['prev_assessment_accuracy'][session_info['prev_assessment_accuracy'] .isna()] = 0

    session_info['mean_session_time'] = session_info['prev_game_time']/session_info['prev_session_count']
    session_info['mean_session_time'][session_info['mean_session_time'] .isna()] = 0

    session_info['mean_game_session_time'] = session_info['prev_game_game_time']/session_info['prev_game']
    session_info['mean_game_session_time'][session_info['mean_game_session_time'] .isna()] = 0

    session_info['mean_assessment_session_time'] = session_info['prev_assessment_game_time']/session_info['prev_assessment']
    session_info['mean_assessment_session_time'][session_info['mean_assessment_session_time'] .isna()] = 0

    session_info['mean_activity_session_time'] = session_info['prev_activity_game_time']/session_info['prev_activity']
    session_info['mean_activity_session_time'][session_info['mean_activity_session_time'] .isna()] = 0

    session_info['mean_event_count'] = session_info['prev_event_count']/session_info['prev_session_count']
    session_info['mean_event_count'][session_info['mean_event_count'] .isna()] = 0

    session_info['mean_game_event_count'] = session_info['prev_game_event_count']/session_info['prev_game']
    session_info['mean_game_event_count'][session_info['mean_game_event_count'] .isna()] = 0
    session_info['mean_assessment_event_count'] = session_info['prev_assessment_event_count']/session_info['prev_assessment']
    session_info['mean_assessment_event_count'][session_info['mean_assessment_event_count'] .isna()] = 0
    session_info['mean_activity_event_count'] = session_info['prev_activity_event_count']/session_info['prev_activity']
    session_info['mean_activity_event_count'][session_info['mean_activity_event_count'] .isna()] = 0


    session_info['avg_time_per_event'] = session_info['mean_session_time']/session_info['mean_event_count']
    session_info['avg_time_per_event'][session_info['avg_time_per_event'] .isna()] = 0


    onehot_title = pd.get_dummies(session_info.title)
    #     onehot_title.columns = ['bird','cart','cauldron', 'chest', 'mushroom']
    session_info = session_info.join(onehot_title)
    
    return(session_info)

In [ ]:
exclusions = ['installation_id', 'game_session', 'type', 'title', 'world',\
           'start_time','end_time','world_sequence', 'rank', 'event_count', 'game_time', 'accurate','num_correct',\
           'inaccurate','num_incorrect', 'Activity', 'Assessment', 'Clip', 'Game', \
           'NONE', 'game_event_count',\
           'activity_event_count', 'assessment_event_count', 'game_session_time',\
           'activity_session_time', 'assessment_session_time', 'game_accurate',\
           'assessment_accurate', 'game_inaccurate', 'assessment_inaccurate','clip_event_count','day_of_week','gap_time','clip_length']

response = 'accuracy_group'

train_session_info = extract_features(train)

del train
gc.collect()

# create features from test
test_session_info = extract_features(test)
del test
gc.collect()

# pick only the latest rows
test_session_info['latest']=test_session_info.groupby(['installation_id'])['start_time'].apply(lambda x: x.rank(ascending =False))
test_features = test_session_info[test_session_info.latest == 1].drop(['latest'],axis = 1)

# pick only completed assessments for labels from test
test_labelled_set = test_session_info[test_session_info.latest != 1].drop(['latest'],axis = 1)
test_labelled_set['accuracy'] =  test_labelled_set.num_correct/(test_labelled_set.num_correct+test_labelled_set.num_incorrect)
test_labelled_set = test_labelled_set[-test_labelled_set.accuracy.isna()]
test_labelled_set['accuracy_group'] = 0
test_labelled_set['accuracy_group'] [test_labelled_set.accuracy == 1] = 3
test_labelled_set['accuracy_group'] [test_labelled_set.accuracy == 0.5] = 2
test_labelled_set['accuracy_group'] [(test_labelled_set.accuracy >0) & (test_labelled_set.accuracy <0.5)] = 1
test_labelled_set = test_labelled_set.drop(['accuracy'],axis =1).drop(exclusions,axis = 1)


train_labelled_set = train_session_info.merge(train_labels[['installation_id','game_session',  response]],\
                                         on = ['installation_id','game_session'],how = 'inner')\
                                 .drop(exclusions,axis = 1)

labelled_set = pd.concat([train_labelled_set,test_labelled_set],sort = False)




# GBM

In [ ]:
 #GBM algorithm
# from sklearn import cross_validation, metrics   #Additional scklearn functions
# from sklearn.model_selection import GridSearchCV
def scale_features(features):
    cols = features.columns
    scaler = MinMaxScaler()
    scaler.fit(features)
    scaled_features = pd.DataFrame(scaler.transform(features))
    scaled_features.columns = cols
    return(scaled_features)





features_train, features_validation, target_train, target_validation = (train_test_split(labelled_set.drop(response,axis =1),
                                                                                labelled_set[response], test_size=0.2, 
                                                                                                 random_state=0))
    
feature_list = list(features_train.columns)
    
# features_train = scale_features(features_train)

gbm = GradientBoostingRegressor(random_state = 0, n_estimators = 5000 , verbose = 1,\
                             n_iter_no_change = 100, learning_rate = 0.01, validation_fraction = 0.2)

gbm.fit(features_train,target_train)

# classifier = GradientBoostingClassifier(n_estimators = 500,learning_rate = 0.1,  random_state = 0,verbose =1, n_iter_no_change =100) 

# classifier.fit(features_train,target_train)

# # preds = classifier.predict(features_validation)
# # preds = np.round(gbm.predict(features_validation),0)

# kappa = cohen_kappa_score(target_validation,preds, weights = 'quadratic')
# kappa






In [ ]:
def classify_accuracy_group(pred, threshold_1, threshold_2,threshold_3):
    pred[pred > threshold_3 ] = 3
    pred[(pred > threshold_2) & (pred <= threshold_3)] = 2
    pred[(pred > threshold_1) & (pred <= threshold_2)] = 1
    pred[pred <= threshold_1] = 0
    return(pred)

In [ ]:

thresh = pd.DataFrame(gbm.predict(features_validation),target_validation).reset_index()
thresh.columns = ['class','val']
print(thresh.groupby(['class'])['val'].mean())
plt.figure(figsize=(10, 6))
sns.distplot(thresh.val[thresh['class'] == 0])
sns.distplot(thresh.val[thresh['class'] == 1])
sns.distplot(thresh.val[thresh['class'] == 2])
sns.distplot(thresh.val[thresh['class'] == 3])


In [ ]:
# def obj_fun(x):
#     preds = classify_accuracy_group(gbm.predict(features_validation),x[0],x[1],x[2])
#     return(1 - cohen_kappa_score(target_validation,preds, weights = 'quadratic'))

# init = [0.5,1.5,2.5]

# res = minimize(obj_fun,init,method='nelder-mead',
#                 options={'xatol': 1e-8, 'disp': True})

# res.x

# bounds = [(0.1,3),(0.1,3),(0.1,3)]

# optimize.shgo(obj_fun, bounds,n=200, iters=5,sampling_method='sobol')



In [ ]:
preds = classify_accuracy_group(gbm.predict(features_validation),0.9999,1.5,2.19)
# preds = np.round(gbm.predict(features_validation),0)

kappa = cohen_kappa_score(target_validation,preds, weights = 'quadratic')
kappa

In [ ]:
preds = classify_accuracy_group(gbm.predict(features_train),0.9999,1.5,2.19)
# preds = np.round(gbm.predict(features_validation),0)

kappa = cohen_kappa_score(target_train,preds, weights = 'quadratic')
kappa

In [ ]:
thresh = pd.DataFrame(gbm.predict(features_train),target_train).reset_index()
thresh.columns = ['class','val']
thresh.groupby(['class'])['val'].mean()


In [ ]:
plt.figure(figsize=(10, 6))
sns.distplot(thresh.val[thresh['class'] == 0])
sns.distplot(thresh.val[thresh['class'] == 1])
sns.distplot(thresh.val[thresh['class'] == 2])
sns.distplot(thresh.val[thresh['class'] == 3])

# LGBM Regressor

In [ ]:
cm = confusion_matrix(target_validation,classify_accuracy_group(gbm.predict(features_validation),0.9999,1.5,2.19)) #labels = target_validation.accuracy_group.unique()
cm
plt.figure(figsize=(10, 6))
sns.heatmap(cm,annot=True,fmt = 'd',cbar=True)


In [ ]:
# cm = confusion_matrix(target_validation,classify_accuracy_group(lgb_reg.predict(features_validation),0.89,1.7,2.21)) #labels = target_validation.accuracy_group.unique()
# cm
# plt.figure(figsize=(10, 6))
# sns.heatmap(cm,annot=True,fmt = 'd',cbar=True)

In [ ]:
importances = list(gbm.feature_importances_)
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)


fig, ax = plt.subplots(figsize=(15,15))

feat_imp = pd.DataFrame(feature_importances,columns = ['Feature','rel_imp'])
ax.barh(feat_imp['Feature'], feat_imp['rel_imp'], align='center')
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('importance')
ax.set_title('Feature importance')


In [ ]:
submission =  pd.DataFrame(test_features['installation_id'])

submission['accuracy_group'] = classify_accuracy_group(gbm.predict(test_features.drop(exclusions,axis = 1)),0.9999,1.5,2.19).astype('int')

submission.to_csv('submission.csv',index=False)

# submission.groupby('accuracy_group').count()

In [ ]:
# # check if there are cases where level2 was played first
# sub = train_session_info.groupby(['installation_id','title'])['session_start'].min().reset_index()

# sub_sub = sub[sub.title == 'Bird Measurer (Assessment)'].merge(sub[sub.title == 'Mushroom Sorter (Assessment)'], \
#                                                                on = ['installation_id'],how = 'inner')\
            
# sub_sub[sub_sub.session_start_x < sub_sub.session_start_y]